In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim


In [8]:
transform = transforms.ToTensor()

train_dataset = torchvision.datasets.FashionMNIST(
    root = "/.data",
    train = True,
    download = True,
    transform = transform)

test_dataset = torchvision.datasets.FashionMNIST(
    root = "./data",
    train = False,
    download = True,
    transform = transform)


100%|██████████| 26.4M/26.4M [00:36<00:00, 721kB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 469kB/s]
100%|██████████| 4.42M/4.42M [00:12<00:00, 359kB/s] 
100%|██████████| 5.15k/5.15k [00:00<00:00, 54.4kB/s]
100%|██████████| 26.4M/26.4M [00:51<00:00, 511kB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 605kB/s]
100%|██████████| 4.42M/4.42M [00:05<00:00, 751kB/s] 
100%|██████████| 5.15k/5.15k [00:00<?, ?B/s]


In [9]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = 32,
    shuffle = True)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = 32,
    shuffle = False)



In [11]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28,128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NeuralNet()



In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)


In [15]:
epochs = 3

for epoch in range(epochs):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}] abgeschlossen")

Epoch [1/3] abgeschlossen
Epoch [2/3] abgeschlossen
Epoch [3/3] abgeschlossen


In [18]:
correct = 0
total = 0


with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct/total
print("Accuracy", accuracy)


Accuracy 86.58


In [20]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

cm = confusion_matrix(y_true, y_pred)
print(cm)

[[832   2  10  18   0   1 133   0   4   0]
 [  1 973   0  19   2   0   4   0   1   0]
 [ 20   1 758  10  56   0 155   0   0   0]
 [ 22  10  10 895  19   0  44   0   0   0]
 [  0   2 100  39 628   0 230   0   1   0]
 [  0   0   0   1   0 956   0  26   1  16]
 [140   1  57  24  26   0 750   0   2   0]
 [  0   0   0   0   0  17   0 954   0  29]
 [  6   0   6   6   2   4  16   5 955   0]
 [  0   0   0   0   0   7   1  35   0 957]]
